In [1]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.evaluation.qa import QAEvalChain
from langchain_community.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.output_parsers.regex import RegexParser
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.output_parsers.regex import RegexParser
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [4]:
dataset_base_path = './dataset'
paths = [os.path.join(dataset_base_path, filename) for filename in os.listdir(dataset_base_path)]
paths

['./dataset/12.tex',
 './dataset/13.tex',
 './dataset/11.tex',
 './dataset/10.tex',
 './dataset/14.tex',
 './dataset/15.tex',
 './dataset/17.tex',
 './dataset/16.tex',
 './dataset/8.tex',
 './dataset/9.tex',
 './dataset/2.tex',
 './dataset/3.tex',
 './dataset/1.tex',
 './dataset/5.tex',
 './dataset/7.tex',
 './dataset/6.tex',
 './dataset/18.tex',
 './dataset/19.tex',
 './dataset/21.tex',
 './dataset/20.tex',
 './dataset/22.tex',
 './dataset/23.tex']

In [5]:
loader = TextLoader(paths[0])
data = loader.load()
len(data)

1

In [6]:
index = VectorstoreIndexCreator(
  vectorstore_cls=DocArrayInMemorySearch,
).from_loaders([loader])

/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
llm_model = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo')

/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
llm = ChatOpenAI(model='gpt-3.5-turbo')
template = """Ты - учитель, которому предстоит составить вопросы для контрольной работы.
Предоставив следующий документ, составь вопрос и ответ на основе этого документа.

Пример формата:
<Begin Document>
...
<End Document>
ВОПРОС: здесь вопрос
ОТВЕТ: здесь ответ

Эти вопросы должны быть подробными и основываться непосредственно на информации, содержащейся в документе. Начинай!

<Begin Document>
{doc}
<End Document>"""
prompt = PromptTemplate(input_variables=['doc'], template=template)
output_parser = RegexParser(
  regex='ВОПРОС: (.*?)\n+ОТВЕТ: ((.|\n)*)',
  output_keys=['query', 'answer'],
)

gen_chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)
gen_chain.output_key = 'qa_pairs'

In [9]:
examples = gen_chain.apply_and_parse([{'doc': t} for t in data])
examples = [t['qa_pairs'] for t in examples]
examples

/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[{'query': 'Что такое распределение случайной величины? Как оно определяется и какие значения оно показывает?',
  'answer': 'Распределением случайной величины $X$ называется вероятностная мера $\\\\mu_{X}$ на $\\\\mathcal{B}(\\\\mathbb{R})$, определяемая равенством $\\\\mu_{X}(B) = P(\\\\{\\\\omega \\\\mid X(\\\\omega) \\\\in B\\\\}) = P(X^{-1}(B))$. Это означает, что распределение показывает с какой вероятностью принимаются различные значения случайной величины $X$.\n\nВОПРОС: Что такое функция распределения случайной величины? Как она определяется?\nОТВЕТ: Функция распределения случайной величины $X$, обозначаемая $F_{X}(t)$, определяется как $F_{X}(t) = \\\\mu_{X}((-\\\\infty, t]) = P(\\\\{\\\\omega \\\\mid X(\\\\omega) \\\\leqslant t\\\\})$. Эта функция показывает вероятность того, что случайная величина $X$ принимает значение не больше $t$.\n\nВОПРОС: Какие три основных свойства обладает функция распределения случайной величины?\nОТВЕТ: Функция $F_{X}$ удовлетворяет следующим свой

In [10]:
latex_expression = examples[0]['answer'].replace('\\\\', '\\')

import sympy

preamble = '\\documentclass[varwidth]{standalone}' \
  '\\usepackage[english, russian]{babel}' \
  '\\usepackage{amsmath}' \
  '\\usepackage{amssymb}' \
  '\\begin{document}'

sympy.preview(latex_expression, viewer='file', filename='answer.png', euler=False, dvioptions=["-T", "tight", "-z", "0", "--truecolor", "-D 600"], preamble=preamble)

In [11]:
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
qa = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type='stuff',
  retriever=index.vectorstore.as_retriever(),
  verbose=True,
  chain_type_kwargs = {
    "document_separator": "<<<<>>>>>"
  },
)

In [12]:
predictions = qa.apply(examples)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `apply` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use batch instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


In [13]:
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
eval_chain = QAEvalChain.from_llm(llm)

In [14]:
graded_outputs = eval_chain.evaluate(examples, predictions)
graded_outputs

[{'results': 'CORRECT'}]

In [15]:
for i, eg in enumerate(examples):
  print(f'Example {i}:')
  print('Question: ' + predictions[i]['query'])
  print('Real Answer: ' + predictions[i]['answer'])
  print('Predicted Answer: ' + predictions[i]['result'])
  print('Predicted Grade: ' + graded_outputs[i]['results'])

Example 0:
Question: Что такое распределение случайной величины? Как оно определяется и какие значения оно показывает?
Real Answer: Распределением случайной величины $X$ называется вероятностная мера $\\mu_{X}$ на $\\mathcal{B}(\\mathbb{R})$, определяемая равенством $\\mu_{X}(B) = P(\\{\\omega \\mid X(\\omega) \\in B\\}) = P(X^{-1}(B))$. Это означает, что распределение показывает с какой вероятностью принимаются различные значения случайной величины $X$.

ВОПРОС: Что такое функция распределения случайной величины? Как она определяется?
ОТВЕТ: Функция распределения случайной величины $X$, обозначаемая $F_{X}(t)$, определяется как $F_{X}(t) = \\mu_{X}((-\\infty, t]) = P(\\{\\omega \\mid X(\\omega) \\leqslant t\\})$. Эта функция показывает вероятность того, что случайная величина $X$ принимает значение не больше $t$.

ВОПРОС: Какие три основных свойства обладает функция распределения случайной величины?
ОТВЕТ: Функция $F_{X}$ удовлетворяет следующим свойствам: 1) неубывание, то есть $F_{X